In [30]:
import numpy as np 
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

1.9.0


### Generate Torque Data and Labels 

In [46]:
N = 100 #Size of train set
x_min = 0
x_max = 10
y_min = -5
y_max = 5

N_closed = 5

#Force Vectors: [(start_x1, start_y1), (end_x1, end_y1),
              #  (start_x2, start_y2), (end_x2, end_y2)]
    
'''
Force Vectors: [start_x1, start_y1, end_x1, end_y1, start_x2, start_y2, end_x2, end_y2]
'''

data = np.random.rand(N, 8) 
data[:,0] *= x_max
data[:,1] *= y_max
data[:,2] *= x_max
data[:,3] *= 0

data[:,4] *= x_max
data[:,5] *= y_min
data[:,6] *= x_max
data[:,7] *= 0

#TODO: Label Data
labels = np.array([i[1] * i[2] + i[5] * i[6] == 0 for i in data]).astype(int)

### Preprocessing
- TODO: Normalize data for input into NN.

In [47]:
print(1 in labels)

False


### Create Model 
- TODO: Create model using NN architecture from tensorflow

In [32]:
print(data.shape)
print(labels.shape)

(100, 8)
(100,)


In [38]:
model = keras.Sequential([
    # keras.layers.Flatten(input_shape=(1,8)),
    keras.layers.Dense(50, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [39]:
model.fit(data, labels, epochs=5)

Epoch 1/5
100/100 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - acc: 0.5000
Epoch 2/5
100/100 [==============================] - 0s 58us/step - loss: 0.0000e+00 - acc: 0.5000
Epoch 3/5
100/100 [==============================] - 0s 53us/step - loss: 0.0000e+00 - acc: 0.5000
Epoch 4/5
100/100 [==============================] - 0s 53us/step - loss: 0.0000e+00 - acc: 0.5000
Epoch 5/5
100/100 [==============================] - 0s 61us/step - loss: 0.0000e+00 - acc: 0.5000


### Small Data Model 
- Low complexity (high bias) model for small data
- Same number of layers as features at least (in this case, 8). 